In [2]:
import pandas as pd
import numpy as np
import sqlalchemy as sql
from dotenv import load_dotenv
import os
import re

In [3]:
load_dotenv()
user = os.getenv('DB_USERNAME')
db = os.getenv('DB')
host = os.getenv('DB_HOST')
password = os.getenv('DB_PASSWORD')

connection_string = f"mysql+pymysql://{user}:{password}@{host}/{db}?charset=utf8mb4"
engine = sql.create_engine(connection_string)


In [4]:
pd.set_option("display.max_columns", None)

In [5]:
#one-hot encode cmc; colors; keywords; produced_mana; 
#average cost per power in set, trade in set, kill in set;
#extract type;
#add trade
#process text (token; replace card name; number of words/char; TF-IDF)
#try to process creature type
#destroy permanent, target sacrifice, life gain, deal damage, loose, draw card (bool or number)

In [6]:
query = """SELECT d.name, scryfall_id, oracle_text, cmc, colors, keywords, type_line, set_name, `set`, d.rarity, `power`, toughness, produced_mana, loyalty, avg_pick FROM scryfall_cards s
INNER JOIN draft_data d ON d.scryfall_id = s.id
WHERE drat_format = 'PremierDraft';
"""

In [7]:
with engine.connect() as conn:
    df = pd.read_sql_query(query, conn)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5714 entries, 0 to 5713
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           5714 non-null   object 
 1   scryfall_id    5714 non-null   object 
 2   oracle_text    5714 non-null   object 
 3   cmc            5714 non-null   object 
 4   colors         5714 non-null   object 
 5   keywords       5714 non-null   object 
 6   type_line      5714 non-null   object 
 7   set_name       5714 non-null   object 
 8   set            5714 non-null   object 
 9   rarity         5714 non-null   object 
 10  power          5714 non-null   object 
 11  toughness      5714 non-null   object 
 12  produced_mana  5714 non-null   object 
 13  loyalty        5714 non-null   object 
 14  avg_pick       5714 non-null   float64
dtypes: float64(1), object(14)
memory usage: 669.7+ KB


In [9]:
df["type_line"].head(10)

0    Legendary Creature — Human Rogue
1         Creature — Phyrexian Cleric
2                  Artifact — Vehicle
3                 Artifact — Treasure
4      Legendary Creature — Rat Pilot
5                             Sorcery
6     Legendary Creature — Frog Beast
7                         Enchantment
8                    Creature — Demon
9            Creature — Spirit Cleric
Name: type_line, dtype: object

In [10]:
df["card_type"] = df["type_line"].apply(lambda x: x.split(" — ")[0])

In [11]:
df["card_type"].head(10)

0    Legendary Creature
1              Creature
2              Artifact
3              Artifact
4    Legendary Creature
5               Sorcery
6    Legendary Creature
7           Enchantment
8              Creature
9              Creature
Name: card_type, dtype: object

In [12]:
df["Legenday"] = df["card_type"].apply(lambda x: "Legendary" in x)

In [13]:
df["card_type"] = df["card_type"].apply(lambda x: x.split(" ")[1] if "Legendary" in x else x)

In [14]:
df["power_*"] = df["power"].apply(lambda x: "*" in x)

In [15]:
df["toughness_*"] = df["toughness"].apply(lambda x: "*" in x)

In [16]:
df[["power_*", "toughness_*"]].head(10)

,power_*,toughness_*
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
5,False,False
6,True,False
7,False,False
8,False,False
9,False,False


In [17]:
def convert_power_tough(x):
    pattern = re.compile(r"\d+")
    if "*" in x:
        match = re.findall(pattern, x)
        if match == []:
            return int(0)
        else:
            return int(match[0])
    else:
        return int(x)

In [18]:
convert_power_tough("12")

12

In [19]:
df["power"] = df["power"].apply(convert_power_tough)

In [20]:
df["toughness"] = df["toughness"].apply(convert_power_tough)

In [21]:
df["power"].head(10)

0    3
1    4
2    4
3    0
4    4
5    0
6    0
7    0
8    2
9    2
Name: power, dtype: int64

In [22]:
df["power_toughness"] = df["power"] + df["toughness"]

In [23]:
df["power_toughness"].head(10)

0    4
1    8
2    7
3    0
4    7
5    0
6    5
7    0
8    4
9    4
Name: power_toughness, dtype: int64

In [24]:
df["cmc"] = df["cmc"].apply(float)

In [25]:
creature_df = df[df["card_type"] == "Creature"]


grouped_sum = creature_df.groupby("set").agg({"power_toughness": "sum", "cmc": "sum"})


average_stat_cost = grouped_sum["cmc"] / grouped_sum["power_toughness"]


df = df.merge(average_stat_cost.rename("average_stat_cost"), how="left", on="set")

In [26]:
df["card_average_stat_cost"] = df["cmc"] / df["power_toughness"]

In [27]:
#highly dubious
df["card_average_stat_cost"] = df["card_average_stat_cost"].apply(lambda x: 1 if x > 100 else x)

In [28]:
df[["scryfall_id", "set", "average_stat_cost", "card_average_stat_cost"]]

,scryfall_id,set,average_stat_cost,card_average_stat_cost
0,f01f12e0-f354-43aa-9e2d-b59a99571a5f,SNC,0.629010,0.750000
1,626c46a3-72b8-4e04-adf2-c9c7aaf94f04,ONE,0.566906,0.625000
2,28273a5b-57b3-4b7a-b017-5886c171c9c9,NEO,0.578534,0.428571
3,2f375674-7ae2-4430-b1f3-c26fa5b201d1,SNC,0.629010,1.000000
4,56a0d43b-4d38-40a7-be6c-8324ab3bf773,NEO,0.578534,0.428571
...,...,...,...,...
5709,9f764be3-dd3f-44b1-a4a6-807d1387590b,M20,0.605351,1.000000
5710,f6b3dd8f-902e-45b8-a422-370705621294,XLN,0.651203,1.000000
5711,74bbe44f-864a-446c-bfc9-5f5188e663b8,M20,0.605351,2.000000
5712,7dd5b0a1-104d-4e0f-82de-65487fbf01ff,WAR,0.603325,0.625000


In [29]:
def count_matching_toughness(row):
    return (sub_df['toughness'] == row['power']).sum()

creature_df = df[df["card_type"] == "Creature"]

temp = pd.DataFrame()
for group, sub_df in creature_df.groupby("set"):
    sub_df["can_trade"] = sub_df.apply(count_matching_toughness, axis=1)
    temp = pd.concat([temp, sub_df[["scryfall_id", "set", "can_trade"]]])
        
df = df.merge(temp, how="left", on=["scryfall_id", "set"])

In [30]:
df[["name", "scryfall_id", "colors", "keywords", "type_line", "set_name", "set"]]

,name,scryfall_id,oracle_text,cmc,colors,keywords,type_line,set_name,set,rarity,power,toughness,produced_mana,loyalty,avg_pick,card_type,Legenday,power_*,toughness_*,power_toughness,average_stat_cost,card_average_stat_cost,can_trade
0,"Toluz, Clever Conductor",f01f12e0-f354-43aa-9e2d-b59a99571a5f,"When Toluz, Clever Conductor enters the battle...",3.0,"['B', 'U', 'W']",['Connive'],Legendary Creature — Human Rogue,Streets of New Capenna,SNC,rare,3,1,0,0,0.000000,Creature,True,False,False,4,0.629010,0.750000,30.0
1,Vivisection Evangelist,626c46a3-72b8-4e04-adf2-c9c7aaf94f04,Vigilance\nCorrupted — When Vivisection Evange...,5.0,"['B', 'W']","['Vigilance', 'Corrupted']",Creature — Phyrexian Cleric,Phyrexia: All Will Be One,ONE,uncommon,4,4,0,0,4.181429,Creature,False,False,False,8,0.566906,0.625000,25.0
2,Mindlink Mech,28273a5b-57b3-4b7a-b017-5886c171c9c9,Flying\nWhenever Mindlink Mech becomes crewed ...,3.0,['U'],"['Flying', 'Crew']",Artifact — Vehicle,Kamigawa: Neon Dynasty,NEO,rare,4,3,0,0,2.379310,Artifact,False,False,False,7,0.578534,0.428571,NaN
3,Courier's Briefcase,2f375674-7ae2-4430-b1f3-c26fa5b201d1,When Courier's Briefcase enters the battlefiel...,2.0,['G'],[],Artifact — Treasure,Streets of New Capenna,SNC,uncommon,0,0,"['B', 'G', 'R', 'U', 'W']",0,0.000000,Artifact,False,False,False,0,0.629010,1.000000,NaN
4,"Greasefang, Okiba Boss",56a0d43b-4d38-40a7-be6c-8324ab3bf773,"At the beginning of combat on your turn, retur...",3.0,"['B', 'W']",[],Legendary Creature — Rat Pilot,Kamigawa: Neon Dynasty,NEO,rare,4,3,0,0,4.980769,Creature,True,False,False,7,0.578534,0.428571,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5709,"Sorin, Imperious Bloodlord",9f764be3-dd3f-44b1-a4a6-807d1387590b,+1: Target creature you control gains deathtou...,3.0,['B'],[],Legendary Planeswalker — Sorin,Core Set 2020,M20,mythic,0,0,0,4,0.000000,Planeswalker,True,False,False,0,0.605351,1.000000,NaN
5710,Overflowing Insight,f6b3dd8f-902e-45b8-a422-370705621294,Target player draws seven cards.,7.0,['U'],[],Sorcery,Ixalan,XLN,mythic,0,0,0,0,0.000000,Sorcery,False,False,False,0,0.651203,1.000000,NaN
5711,Voracious Hydra,74bbe44f-864a-446c-bfc9-5f5188e663b8,Trample\nVoracious Hydra enters the battlefiel...,2.0,['G'],"['Trample', 'Fight']",Creature — Hydra,Core Set 2020,M20,rare,0,1,0,0,0.000000,Creature,False,False,False,1,0.605351,2.000000,1.0
5712,Ashiok's Skulker,7dd5b0a1-104d-4e0f-82de-65487fbf01ff,{3}{U}: Ashiok's Skulker can't be blocked this...,5.0,['U'],[],Creature — Nightmare,War of the Spark,WAR,common,3,5,0,0,0.000000,Creature,False,False,False,8,0.603325,0.625000,19.0


In [31]:
df["can_trade"] = df["can_trade"].fillna(0)

In [63]:
def one_hot_expand(col, prefix, sep=",", clean=True):
    if clean:
        df[col] = df[col].apply(lambda x: x[1:-1])
        df[col] = df[col].apply(lambda x: x.replace("'", ""))
        df[col] = df[col].apply(lambda x: x.replace(" ", ""))
    df2 = df[col].str.split(sep, expand=True)
    stacked = df2.stack()
    one_hot_encoded = pd.get_dummies(stacked, prefix=prefix).groupby(level=0).sum()
    return df.join(one_hot_encoded)

In [33]:
df["colors"] = df["colors"].apply(lambda x: x[1:-1])


In [34]:
df["colors"] = df["colors"].apply(lambda x: x.replace("'", ""))
df["colors"] = df["colors"].apply(lambda x: x.replace(" ", ""))

In [35]:
df["colors"].unique()

array(['B,U,W', 'B,W', 'U', 'G', 'B', 'B,G', 'W', 'G,U', '', 'U,W', 'R',
       'G,W', 'G,R', 'R,W', 'B,G,R', 'R,U', 'B,U', 'B,R', 'G,U,W',
       'B,R,U', 'G,R,W', 'B,G,R,U,W', 'B,R,W', 'B,G,U,W', 'R,U,W',
       'B,G,U', 'G,R,U', 'B,G,W', 'G,R,U,W'], dtype=object)

In [36]:
df2 = df['colors'].str.split(',', expand=True)

In [37]:
stacked = df2.stack()

In [38]:
one_hot_encoded = pd.get_dummies(stacked, prefix='colors').groupby(level=0).sum()

In [39]:
df = df.join(one_hot_encoded)

In [40]:
df = df.rename(columns={"colors_":"uncolored"})

In [41]:
df[df["uncolored"] == 1]

,name,scryfall_id,oracle_text,cmc,colors,keywords,type_line,set_name,set,rarity,power,toughness,produced_mana,loyalty,avg_pick,card_type,Legenday,power_*,toughness_*,power_toughness,average_stat_cost,card_average_stat_cost,can_trade,uncolored,colors_B,colors_G,colors_R,colors_U,colors_W
13,Phyrexian Dragon Engine,e2b826be-4256-4fd6-ad4d-6c80933ee940,Double strike\nWhen Phyrexian Dragon Engine en...,3.0,,"['Unearth', 'Double strike']",Artifact Creature — Phyrexian Dragon,The Brothers' War,BRO,rare,2,2,0,0,1.306364,Artifact Creature,False,False,False,4,0.612971,0.750000,0.0,1,0,0,0,0,0
15,Goring Warplow,9d927dfd-ae66-4235-a61b-39c85d0c1222,Prototype {1}{B} — 1/1 (You may cast this spel...,6.0,,"['Prototype', 'Deathtouch']",Artifact Creature — Construct,The Brothers' War,BRO,common,5,4,0,0,7.329591,Artifact Creature,False,False,False,9,0.612971,0.666667,0.0,1,0,0,0,0,0
16,Azusa's Many Journeys,e8a51d2a-1582-4bad-995c-e7fe9f810149,0,2.0,,['Transform'],Enchantment — Saga // Enchantment Creature — H...,Kamigawa: Neon Dynasty,NEO,uncommon,0,0,0,0,5.030011,Enchantment,False,False,False,0,0.578534,1.000000,0.0,1,0,0,0,0,0
24,The Mycosynth Gardens,5a931463-25f6-4e31-95b4-bb4a9388009b,"{T}: Add {C}.\n{1}, {T}: Add one mana of any c...",0.0,,[],Land — Sphere,Phyrexia: All Will Be One,ONE,rare,0,0,"['B', 'C', 'G', 'R', 'U', 'W']",0,4.958002,Land,False,False,False,0,0.566906,NaN,0.0,1,0,0,0,0,0
28,Ecologist's Terrarium,1d981026-b2df-4d8d-a9b4-296b011d9925,When Ecologist's Terrarium enters the battlefi...,2.0,,[],Artifact,Kamigawa: Neon Dynasty,NEO,common,0,0,0,0,7.280433,Artifact,False,False,False,0,0.578534,1.000000,0.0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5664,Thaumatic Compass,392af78e-34d5-4b1b-8b29-0e702271e4d7,0,2.0,,['Transform'],Artifact // Land,Ixalan,XLN,rare,0,0,['C'],0,0.000000,Artifact // Land,False,False,False,0,0.651203,1.000000,0.0,1,0,0,0,0,0
5668,Forebear's Blade,52212fd5-551e-4bc1-9dac-6361e27c27ad,Equipped creature gets +3/+0 and has vigilance...,3.0,,['Equip'],Artifact — Equipment,Dominaria,DOM,rare,0,0,0,0,0.000000,Artifact,False,False,False,0,0.610132,1.000000,0.0,1,0,0,0,0,0
5679,Hinterland Harbor,d631d040-51e9-4540-91e7-aeb5ade84090,Hinterland Harbor enters the battlefield tappe...,0.0,,[],Land,Dominaria,DOM,rare,0,0,"['G', 'U']",0,0.000000,Land,False,False,False,0,0.610132,NaN,0.0,1,0,0,0,0,0
5693,Watery Grave,7d4595f2-9297-40dc-b2dd-7144bbb401f7,({T}: Add {U} or {B}.)\nAs Watery Grave enters...,0.0,,[],Land — Island Swamp,Guilds of Ravnica,GRN,rare,0,0,"['B', 'U']",0,0.000000,Land,False,False,False,0,0.651246,NaN,0.0,1,0,0,0,0,0


In [42]:
df = one_hot_expand("keywords", "keywords")

In [43]:
df = one_hot_expand("produced_mana", "produced_mana")

In [44]:
df = df.drop(columns=["produced_mana_", "keywords_"])

In [45]:
set_df_kw = df

In [49]:
single_word = "\w+"
single_word_hypen = "\w+-\w+"
regex_1 = f"({single_word}|{single_word_hypen})"

In [53]:
set_df_kw['tapping_ability'] = np.where(set_df_kw['oracle_text'].str.contains('{T}'),1,0)

# Includes multiple choice

# We create a column that is 1 if the card has '• ' in the oracle_text
set_df_kw['multiple_choice'] = np.where(set_df_kw['oracle_text'].str.contains('• '),1,0)

In [54]:
try:
    set_df_kw[['face','back']] = set_df_kw['type_line'].str.split(' // ',expand=True).fillna('None')
except:
    set_df_kw[['face','back']] = [set_df_kw['type_line'],"None"]

# We then separate the face type using the "-" as separator
try:
    set_df_kw[['face_type','face_subtype']] = set_df_kw['face'].str.split(' — ',expand=True).fillna('None')
except:
    set_df_kw['face_type'] = set_df_kw['face']
    set_df_kw['face_subtype'] = "None"

# If a card has a back, we then separate the back type using the "-" as separator
try:
    set_df_kw[['back_type','back_subtype']] = set_df_kw['back'].str.split(' — ',expand=True).fillna('None')
except:
    set_df_kw['back_type'] = set_df_kw['back']
    set_df_kw['back_subtype'] = "None"

In [ ]:
try:
    set_df_kw['counterspell'] = np.where((set_df_kw['counter_tkn']==1) &

                                            ((set_df_kw['oracle_text'].str.lower().str.contains("counter target")) |
                                            (set_df_kw['oracle_text'].str.lower().str.contains("counter all")) |
                                            (set_df_kw['oracle_text'].str.lower().str.contains("counter it")))
                                            ,1,0)
except:
    set_df_kw['counterspell'] = 0

set_df_kw['manarock'] = np.where(
                                ((set_df_kw['tapping_ability']==1) |
                                    (set_df_kw['oracle_text']).str.lower().str.contains("tap")) &

                                (set_df_kw['type_line']).str.lower().str.contains("artifact") &

                                (
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"{t}: add.*?(mana of any color|mana of that color|{(.*?)})",regex=True)==True) |
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"{t}, tap an untapped.*?(mana of any color|mana of that color|{(.*?)})",regex=True)==True) |
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"{t}: choose a color",regex=True)==True)
                                )
                                ,1,0)

set_df_kw['manadork'] = np.where(
                                (set_df_kw['tapping_ability']==1)&
                                (set_df_kw['manarock']!=1) &
                                (set_df_kw['back_type']!="Land") &
                                (set_df_kw['type_line']).str.lower().str.contains("creature") &

                                (
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"{t}: add",regex=True)==True) |
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"{t}:.*?add one mana",regex=True)==True) |
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"{t}: add",regex=True)==True) |
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"add (one|two|three|four|five) mana",regex=True)==True)
                                )
                                ,1,0)


set_df_kw['removal'] = np.where(
                                    (
                                    (set_df_kw['oracle_text'].str.lower().str.contains(f"(destroy|exile) target ({regex_1}|({regex_1}, {regex_1})|({regex_1}, {regex_1}, {regex_1})|({regex_1}, {regex_1}, {regex_1}, {regex_1})) (creature|permanent)(?! from (a|your) graveyard| card from (a|your) graveyard)",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"(destroy|exile) another target (creature|permanent)",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"destroy any number of target (creature|creatures|permanent|permanents)",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"(destroy|exile) target (attacking|blocking|attacking or blocking) creature",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"destroy up to (one|two|three) target (\w+) (creature|permanent|creatures|permanents)",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"exile up to (one|two|three) target (creature|permanent|creatures|permanents)",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"exile up to (one|two|three) target (nonland|nonartifact) (creature|permanent|creatures|permanents)",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"exile up to (one|two|three) target (\w+) (\w+) (creature|permanent|creatures|permanents)",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"(destroy|exile) target (\w+) or creature",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"(destroy|exile) a (creature|permanent) with the (greatest|highest|lowest) (power|toughness|converted mana cost|mana value)",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"(destroy|exile) target (creature|permanent)(?! from a graveyard| card from a graveyard)", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"(destroy|exile) up to (\w+) target (attacking or blocking|attacking|blocking) (creature|creatures)", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"target (player|opponent) sacrifices a (creature|permanent)",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"each (player|opponent) sacrifices (a|one|two|three|four) (creature|creatures|permanent|permanents)",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"enchanted (creature|permanent) is a treasure",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"enchanted creature doesn't untap",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"(annihilator)")==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"deals damage equal to its power to target creature",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"(fights|fight) target creature")==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"(those creatures|the chosen creatures) fight each other",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"(fights|fight) up to (\w+) target (creature|creatures)", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"(fights|fight) another target creature",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"choose target creature you don't control.*?each creature.*?deals damage equal.*?to that creature",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"you may have (cardname|it) fight (that creature|target creature|another target creature)",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"target creature deals damage to itself equal to (its power)",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"target creature gets -[0-9]/-[2-9]", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"target creature gets \+[0-9]/-[2-9]", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"target creature an opponent controls gets \-[0-9]/\-[2-9]", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"enchanted creature (gets|has).*?loses (all|all other) abilities", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"enchanted creature gets \-[0-9]/\-[2-9]", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"enchanted creature gets \-[0-9]/\-[2-9]", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"enchanted creature gets \+[0-9]/\-[2-9]", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"(enchanted|target) creature gets \-[0-9][0-9]/\-[0-9][0-9]", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains("target creature gets \-x/\-x")==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains("target creature gets \+x/\-x")==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"target creature an opponent controls gets \-x/\-x", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"enchanted creature gets \-x/\-x", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"enchanted (creature|permanent) can't attack or block",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains("enchanted creature has defender")==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains("enchanted creature can't block.*?its activated abilities can't be activated")==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"enchanted creature.*?loses all abilities",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"enchanted (creature|permanent) can't attack.*?block.*?and its activated abilities can't be activated", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"deals ([2-9|x]) damage.*?(creature|any target|divided as you choose|to each of them)", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"deals ([2-9|x]) damage.*?to each of up to (one|two|three|four) (target|targets)", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"deals damage equal to.*?to (any target|target creature|target attacking creature|target blocking creature|target attacking or blocking creature)", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"target creature deals (.*?) damage to itself", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"deals damage to (any target|target creature|target attacking creature|target blocking creature|target attacking or blocking creature).*?equal to", regex=True)==True)) &

                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"(cardname|it) deals [a-zA-Z0-9] damage to that player.",regex=True)==False) &
                                    (set_df_kw['oracle_text'].str.lower().str.contains("(cardname|it) deals [a-zA-Z0-9] damage to target (player|opponent) or planeswalker")==False) &
                                    (set_df_kw['oracle_text'].str.lower().str.contains("(cardname|it) deals [a-zA-Z0-9] damage to that creature's controller")==False) &
                                    (set_df_kw['oracle_text'].str.lower().str.contains("that was dealt damage this turn")==False) &
                                    (set_df_kw['oracle_text'].str.lower().str.contains("^(?!damage|creature)\w* random")==False) &
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"search.*?(creature|artifact|enchantment) card",regex=True)==False) &
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"(destroy|exile) target land",regex=True)==False) &
                                    (set_df_kw['oracle_text'].str.lower().str.contains("return it to the battlefield")==False) &
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"return that (card|creature|permanent) to the battlefield",regex=True)==False) &
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"if you control a.*?^(?!liliana)\w* planeswalker",regex=True)==False) &
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"^(?!additional cost|additional cost)\w* exile (target|a|one|two|three|all).*?from (your|a|target opponent's) graveyard",regex=True)==False)
                                    ,1,0)

set_df_kw['wrath'] = np.where(
                                (
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"(destroy|exile) (all|all other|each|each other|all attacking) (creature|creatures|(.*?) creatures|permanent|permanents|(.*?) permanents|(nonland|multicolored) permanent|(nonland|multicolored) permanents)",regex=True)==True) |
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"each (creature|other creature) gets -(x|[0-9])/-(x|[2-9])", regex=True)==True) |
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"each creature deals damage to itself equal to", regex=True)==True) |
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"(destroy|exile) all artifacts, creatures, and enchantments", regex=True)==True) |
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"sacrifices (all|that many) (creatures|(.*?) creatures|permanents|(.*?) permanents)", regex=True)==True) |
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"chooses.*?then sacrifices the rest", regex=True)==True) |
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"creatures.*?get -(x|[0-9])/-(x|[2-9])", regex=True)==True) | #Crippling Fear
                                (set_df_kw['oracle_text'].str.lower().str.contains(f"deals ([3-9]|x|[1-9][0-9]) damage to each (creature|{regex_1} creature)", regex=True)==True)
                                )
                                ,1,0)

regex_2 = "(land|lands|basic land|basic lands|plains|island|swamp|mountain|forest|plains|islands|swamps|mountains|forests|basic plains|basic island|basic swamp|basic mountain|basic forest|basic plains|basic islands|basic swamps|basic mountains|basic forests)"

regex_3 = "(a|one|one|two|three|up to one|up to two|up to three|up to ten|up to x|x)"

set_df_kw['ramp'] = np.where(
                            (set_df_kw['face_type']!="Land") &
                            (set_df_kw['manadork']!=1) &
                            (set_df_kw['manarock']!=1) &
                            (set_df_kw['face_type']!="Snow Land") &
                            (set_df_kw['face_type']!="Artifact Land") &
                            (set_df_kw['type_line'].str.lower().str.contains(r"(\w+) // land", regex=True)==False) &
                            (
                            (set_df_kw['oracle_text'].str.lower().str.contains(r"{t}: add", regex=True)==True) |
                            (set_df_kw['oracle_text'].str.lower().str.contains(r"^(?!{[1-9]}: )\w* add (one|two) mana", regex=True)==True) |
                            (set_df_kw['oracle_text'].str.lower().str.contains(r"{[1]}, {t}: add ({(c|w|u|b|r|g)}{(c|w|u|b|r|g)}|two)", regex=True)==True) |
                            (set_df_kw['oracle_text'].str.lower().str.contains(r"whenever enchanted land is tapped for mana.*?adds", regex=True)==True) |
                            (set_df_kw['oracle_text'].str.lower().str.contains(f"search (your|their) library for {regex_3} {regex_2}.*?put.*?onto the battlefield", regex=True)==True)
                            ) &
                            (
                            (set_df_kw['oracle_text'].str.lower().str.contains(r"{[1-9]}, {t}: add one mana", regex=True)==False) &
                            (set_df_kw['oracle_text'].str.lower().str.contains(r"enchanted land.*?{t}: add {(c|1|w|u|b|r|g)}", regex=True)==False) &
                            (set_df_kw['oracle_text'].str.lower().str.contains(r"destroy target (land|nonbasic land)", regex=True)==False) &
                            (set_df_kw['oracle_text'].str.lower().str.contains(r"spend this mana only to", regex=True)==False)
                            )
                            ,1,0)

set_df_kw['tutor'] = np.where(
                            (set_df_kw['ramp']!=1) &
                            (set_df_kw['face_type']!="Land") &
                            (set_df_kw['face_type']!="Snow Land") &
                            (set_df_kw['face_type']!="Artifact Land") &
                            (
                            (set_df_kw['oracle_text'].str.lower().str.contains(r"search your (library|library and graveyard) for (a|an|any|any one|one|up to one|two|up to two|three|up to three|four|up to four|a(white|blue|black|red|green|colorless)) (card|cards|permanent|permanents|equipment|aura|aura or equipment|legendary|enchantment|enchantments|artifact|artifacts|creature|(.*?) creature cards|creature cards|creatures|sorcery|sorceries|instant|instants|planeswalker)", regex=True)==True)
                            ) &
                            (
                            (set_df_kw['oracle_text'].str.lower().str.contains(r"put (it|them|those cards|that card) into your graveyard", regex=True)==False) &
                            (set_df_kw['oracle_text'].str.lower().str.contains(r"named", regex=True)==False)
                            )
                            ,1,0)

set_df_kw['cardraw'] = np.where(
                                (
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"draw (a|one|two|three|four|five|six|seven|x|(.*?) x) (card|cards)", regex=True)==True) |
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"draw (cards equal to|that many cards)", regex=True)==True) |
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"target player draws (.*?) (card|cards)", regex=True)==True) |
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"(look at|reveal) the.*?put.*?(into|in) your hand", regex=True)==True) |
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"(exile|look at the).*?(card|cards).*?you may (cast|play)", regex=True)==True)
                                ) &
                                (
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"whenever you draw a card", regex=True)==False) &
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"if you would draw a card", regex=True)==False) &
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"draw (a|one|two|three|four) (card|cards), then discard (a|one|two|three|four) (card|cards)", regex=True)==False) &
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"discard (a|one|two|three|four) (card|cards), then draw (a|one|two|three|four) (card|cards)", regex=True)==False)
                                )
                                ,1,0)

set_df_kw['burn'] = np.where(
                            (
                            (set_df_kw['oracle_text'].str.lower().str.contains(r"deals ([1-9|x]) damage.*?(any target|player|opponent|to them|to each of them)", regex=True)==True) |
                            (set_df_kw['oracle_text'].str.lower().str.contains(r"deals (x|two|three|four|five) times (damage|x damage).*?(any target|player|opponent|to them|to each of up to)", regex=True)==True) |
                            (set_df_kw['oracle_text'].str.lower().str.contains(r"deals damage equal to.*?to (any target|target player|target opponent|to them|each player|each opponent)", regex=True)==True) |
                            (set_df_kw['oracle_text'].str.lower().str.contains(r"deals damage to (any target|target player|target opponent|to them|each player|each opponent).*?equal to", regex=True)==True) |
                            (set_df_kw['oracle_text'].str.lower().str.contains(r"deals that much damage to (any target|target player|target opponent|each player|each opponent|that source's controller)", regex=True)==True)
                            )
                            ,1,0)

set_df_kw['discard'] = np.where(
                                (
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"(that|target|each) (player|opponent) discards (a|one|two|three|that|all|all the) (card|cards)", regex=True)==True) |
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"unless that player.*?discards a card", regex=True)==True) |
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"target (player|opponent) reveals their hand.*?you choose.*?exile (that|it)", regex=True)==True)
                                )
                                ,1,0)

set_df_kw['enters_bf'] = np.where(
                                (
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"(enter|enters) the battlefield", regex=True)==True)
                                )
                                &
                                (
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"(enter|enters) the battlefield tapped", regex=True)==False) &
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"land (enter|enters) the battlefield", regex=True)==False) &
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"it becomes day", regex=True)==False) &
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"enchant creature", regex=True)==False)
                                )
                                ,1,0)

set_df_kw['die_trigger'] = np.where(
                                    (
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"when (cardname|equipped creature) dies", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"whenever.*?dies", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"whenever.*?you (control|don't control) dies", regex=True)==True)
                                    )
                                    ,1,0)

set_df_kw['attack_trigger'] = np.where(
                                        (
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"(when|whenever) (cardname|equipped creature|it) attacks", regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"(when|whenever) (cardname|equipped creature|it) and.*?(other|another) (creature|creatures) attack", regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"(battalion|exert|raid)", regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"(when|whenever) (cardname|equipped creature|it) enters the battlefield or attacks", regex=True)==True)
                                        )
                                        ,1,0)

set_df_kw['pseudo_ramp'] = np.where(
                                    (
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"you may put a (land|basic land).*?onto the battlefield", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"(you|each player) may (play|put) an additional land", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"if it's a land card, you may put it onto the battlefield", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"sacrifice.*?add.*?({(.*?)}|to your mana pool|mana of (any|any one) color)", regex=True)==True)
                                    )
                                    ,1,0)

set_df_kw['static_ramp'] = np.where(
                                    ((set_df_kw['type_line'].str.lower().str.contains("enchantment")) |
                                    (set_df_kw['type_line'].str.lower().str.contains("creature")) |
                                    (set_df_kw['type_line'].str.lower().str.contains("artifact"))) &
                                    (set_df_kw['back'].str.lower().str.contains("land")==False) &
                                    (
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"at the beginning of.*?add.*?(mana|{(.*?)})", regex=True)==True)
                                    )
                                    ,1,0)

regex_4 = "(a|one|up to one|two|up to two|three|up to three|four|up to four|five|up to five|six|up to six|x|up to x)"

set_df_kw['creature_tokens'] = np.where(
                                        (
                                        (set_df_kw['oracle_text'].str.lower().str.contains(f"(create|put) {regex_4}.*?creature (token|tokens)", regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(f"(living weapon|amass|fabricate|afterlife|populate)", regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"(creature tokens|creature tokens with.*?) are created instead", regex=True)==True)
                                        )
                                        ,1,0)

set_df_kw['extra_turn'] = np.where(set_df_kw['oracle_text'].str.lower().str.contains(r"(take|takes) (an|one|two) extra (turn|turns)", regex=True)==True
                                    ,1,0)

set_df_kw['plus1_counters'] = np.where(
                                        (
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"\+1/\+1 (counter|counters)", regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"(evolve|mentor|adapt|bolster|bloodthirst|devour|monstrosity|reinforce|training)", regex=True)==True)
                                        )
                                        ,1,0)

set_df_kw['graveyard_hate'] = np.where(
                                        (
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"exile.*?from (graveyards|a graveyard|his or her graveyard|target player's graveyard|each opponent's graveyard)", regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"remove all graveyards from the game", regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"exile.*?all (cards|creatures) from all (graveyards|opponents' hands and graveyards)", regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"exile each opponent's graveyard", regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"if a.*?(card|creature|permanent) would (be put into.*?graveyard|die).*?(instead exile|exile it instead)", regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"choose target card in (target opponent's|a) graveyard.*?exile (it|them)", regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"(target|each) player puts all the cards from their graveyard", regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"(creature cards|permanents|creatures|permanent cards) in (graveyards|graveyards and libraries) can't enter the battlefield", regex=True)==True)
                                        )
                                        ,1,0)

set_df_kw['free_spells'] = np.where(
                                    (
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"(rather than pay|without paying) (its|it's|their|this spell's|the) mana cost", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"cascade", regex=True)==True)
                                    )
                                    &
                                    (
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"you may pay {", regex=True)==False)
                                    )
                                    ,1,0)

set_df_kw['bounce_spell'] = np.where(
                                    (
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"return.*?to (it's|its|their) (owner's|owners') (hand|hands)", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"owner.*?puts it.*?(top|bottom).*?library", regex=True)==True)
                                    )
                                    &
                                    (
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"^(?!islands)\w* you control", regex=True)==False) &
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"(when|whenever).*?dies.*?return.*?to its owner's hand", regex=True)==False) &
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"return (cardname|the exiled card) to its owner's hand", regex=True)==False) &
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"whenever cardname.*?return it to its owner's hand", regex=True)==False)
                                    )
                                    ,1,0)

set_df_kw['sac_outlet'] = np.where(
                                    (
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"sacrifice (a|another) (creature|permanent)", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"(exploit)", regex=True)==True)
                                    )
                                    ,1,0)

set_df_kw['sac_payoff'] = np.where(
                                    (
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"whenever (you|a player) (sacrifice|sacrifices) a (creature|permanent)", regex=True)==True)
                                    )
                                    ,1,0)

set_df_kw['cant_counter'] = np.where(
                                    (
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"can't be countered", regex=True)==True)
                                    )
                                    ,1,0)

set_df_kw['costx_more'] = np.where(
                                    (
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"(cost|costs) (.*?) more to cast", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"ward", regex=True)==True)
                                    )
                                    ,1,0)

set_df_kw['costx_moreactivate'] = np.where(
                                            (
                                            (set_df_kw['oracle_text'].str.lower().str.contains(r"(cost|costs) (.*?) more to activate", regex=True)==True)
                                            )
                                            ,1,0)

set_df_kw['costx_less'] = np.where(
                                    (
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"(cost|costs) (.*?) less to cast", regex=True)==True)
                                    )
                                    ,1,0)

set_df_kw['costx_lessacitivate'] = np.where(
                                            (
                                            (set_df_kw['oracle_text'].str.lower().str.contains(r"(cost|costs) (.*?) less to activate", regex=True)==True)
                                            )
                                            ,1,0)

set_df_kw['whenever_opp'] = np.where(
                                    (
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"whenever (an opponent|a player)", regex=True)==True)
                                    )
                                    ,1,0)

regex_5 = "(all|each|another|another target|x|x target|a|target|any number of|one|up to one|up to one target|two|up to two|up to two target|three|up to three|up to three target|four|up to four|up to four target)"
regex_6 = "(card|cards|creature|creatures|nonlegendary creature|creature card|creature cards|permanent|permanents|permanent card|permanent cards|land|lands|land card|land cards|instant or sorcery card|equipment card|aura card|aura or equipment card|artifact or enchantment)"

set_df_kw['returnfrom_gy'] = np.where(
                                        (
                                        (set_df_kw['oracle_text'].str.lower().str.contains(fr"return {regex_5} {regex_6}.*?from your graveyard to your hand", regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(fr"return cardname from your graveyard to your hand", regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(fr"choose.*?graveyard.*?return.*?to your hand", regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(fr"return.*?up to.*?from your graveyard to your hand", regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(fr"return (target|another target).*?card from your graveyard to your hand", regex=True)==True)
                                        )
                                        ,1,0)

set_df_kw['reanimation'] = np.where(
                                    (
                                    (set_df_kw['oracle_text'].str.lower().str.contains(fr"(return|put) {regex_5} {regex_6}.*?from (your|a) graveyard (to|onto) the battlefield", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(fr"return cardname from your graveyard to the battlefield", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(fr"choose.*?graveyard.*?return.*?to the battlefield", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(fr"return.*?up to.*?from your graveyard to the battlefield", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(fr"enchant creature card in (a|your) graveyard.*?return enchanted creature card to the battlefield under your control", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(fr"(return|returns|put) (all|any number of) (creature|permanent|enchantment|artifact|legendary permanent|legendary creature|nonlegendary creature|nonlegendary permanents|(.*?), (.*?) and (.*?)) cards.*?from (their|your|all) (graveyard|graveyards) (to|onto) the battlefield", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(fr"(return|put) (target|another target).*?card from your graveyard to the battlefield", regex=True)==True)
                                    )
                                    ,1,0)

set_df_kw['castfrom_gy'] = np.where(
                                    (
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"you may cast cardname from your graveyard", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"flashback {", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"jump-start", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"escape—{", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"disturb {", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"unearth {", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"retrace", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"embalm", regex=True)==True)
                                    )
                                    ,1,0)

set_df_kw['lord'] = np.where(
                            (
                            (set_df_kw['type_line'].str.lower().str.contains("creature")) |
                            (set_df_kw['type_line'].str.lower().str.contains("artifact")) |
                            (set_df_kw['type_line'].str.lower().str.contains("enchantment"))
                            ) &
                            (
                            (set_df_kw['oracle_text'].str.lower().str.contains(r"get \+[1-9]/\+[0-9]", regex=True)==True) |
                            (set_df_kw['oracle_text'].str.lower().str.contains(r"(battle cry)", regex=True)==True) |
                            (set_df_kw['oracle_text'].str.lower().str.contains(r"each (creature|other creature).*?gets \+[1-9]/\+[0-9]", regex=True)==True)
                            )
                            &
                            (
                            (set_df_kw['oracle_text'].str.lower().str.contains(r"until end of turn", regex=True)==False)
                            )
                            ,1,0)

set_df_kw['upkeep_trigger'] = np.where(
                                        (
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"beginning of (your|enchanted player's|each|each player's) upkeep", regex=True)==True)
                                        )
                                        &
                                        (
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"beginning of your upkeep, sacrifice cardname", regex=True)==False)
                                        )
                                        ,1,0)

set_df_kw['endstep_trigger'] = np.where(
                                        (
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"beginning of (your|enchanted player's|each) end step", regex=True)==True)
                                        )
                                        &
                                        (
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"sacrifice.*? at the beginning of your end step", regex=True)==False)
                                        )
                                        ,1,0)

set_df_kw['landfall'] = np.where(
                                (
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"whenever a land enters the battlefield under your control", regex=True)==True) |
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"landfall", regex=True)==True)
                                )
                                ,1,0)

set_df_kw['combat_trigger'] = np.where(
                                        (
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"beginning of (combat|each combat)", regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"deals combat damage", regex=True)==True)
                                        )
                                        ,1,0)

set_df_kw['life_gain'] = np.where(
                                (
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"gain (.*?) life", regex=True)==True) |
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"gains (.*?) x life", regex=True)==True) |
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"gain life equal", regex=True)==True) |
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"(lifelink|extort)", regex=True)==True)
                                )
                                ,1,0)

set_df_kw['treasure_tokens'] = np.where(
                                        (
                                        (set_df_kw['oracle_text'].str.lower().str.contains(f"(create|put) {regex_4}.*?treasure (token|tokens)", regex=True)==True)
                                        )
                                        ,1,0)

set_df_kw['protection'] = np.where(
                                    (
                                    (set_df_kw['oracle_text'].str.lower().str.contains(f"(hexproof|ward|indestructible|shroud)", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(f"can't (be|become) (the|target)", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(f"protection from", regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(f"becomes the target of a spell", regex=True)==True)
                                    )
                                    &
                                    (
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"becomes the target of.*?sacrifice (it|cardname)", regex=True)==False) &
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"becomes the target of.*?shuffle.*?into its owner's library", regex=True)==False) &
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"becomes.*?with hexproof.*?until end of turn", regex=True)==False)
                                    )
                                    ,1,0)

set_df_kw['cost_reduction'] = np.where(
                                        (
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"(delve|convoke|affinity|foretell|madness|miracle|spectacle)", regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"evoke", regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"you may pay.*?to cast this spell", regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"you may pay (.*?) rather than pay", regex=True)==True)
                                        )
                                        ,1,0)

set_df_kw['mana_multipliers'] = np.where(
                                        (
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"(whenever|if).*?tap (a|an) (land|permanent|nonland permanent|plains|island|swamp|mountain|forest|creature) for mana.*?add (one mana|an additional|{(.*?)})", regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"(whenever|if).*?tap (a|an) (land|permanent|nonland permanent|plains|island|swamp|mountain|forest|creature) for mana.*?it produces.*?instead", regex=True)==True)
                                        )
                                        ,1,0)

set_df_kw['card_selection'] = np.where(
                                        (
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"scry", regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"look at the top.*?bottom of your library.*?on top", regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"look at the top.*?on top.*?bottom of your library", regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"look at the top.*?graveyard.*?on top", regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"look at the top.*?on top.*?graveyard", regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"look at the top.*?you may put.*?into your graveyard", regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"surveil", regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"(explore|explores)", regex=True)==True)
                                        )
                                        &
                                        (
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"whenever a creature you control explores", regex=True)==False)
                                        )
                                        ,1,0)

set_df_kw['whenever_cast'] = np.where(
                                        (
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"(whenever you cast|prowess)",regex=True)==True)
                                        )
                                        &
                                        (
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"you may transform", regex=True)==False)
                                        )
                                        ,1,0)

set_df_kw['gain_control'] = np.where(
                                    (
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"gain control of",regex=True)==True)
                                    )
                                    ,1,0)

set_df_kw['unblockeable'] = np.where(
                                    (set_df_kw['type_line'].str.lower().str.contains("creature")) &
                                    (
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"(cardname|you control) can't be blocked",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"(unblockable|shadow)",regex=True)==True)
                                    )
                                    &
                                    (
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"cardname can't be blocked by", regex=True)==False)
                                    )
                                    ,1,0)

set_df_kw['difficult_block'] = np.where(
                                        (set_df_kw['type_line'].str.lower().str.contains("creature")) &
                                        (
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"cardname can't be blocked by",regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"(menace|first strike|flying|deathtouch|double strike|fear|intimidate)",regex=True)==True)
                                        )
                                        ,1,0)

set_df_kw['create_copy'] = np.where(
                                    (
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"create a copy of",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"(that's|as) a copy of",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"copy (target|it|them|that spell|that ability)",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"you may copy",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"(storm)",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"becomes a copy",regex=True)==True)
                                    )
                                    ,1,0)

set_df_kw['milling'] = np.where(
                                (
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"(mill|mills)",regex=True)==True) |
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"puts the top.*?of (their|his or her|your) library into (their|his or her|your) graveyard",regex=True)==True) |
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"exile the top (.*?) cards of (target|each) (player|opponent|players|opponents)",regex=True)==True) |
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"(target|each) opponent exiles cards from the top of their library",regex=True)==True)
                                )
                                ,1,0)

set_df_kw['trigger_multiplier'] = np.where(
                                            (
                                            (set_df_kw['oracle_text'].str.lower().str.contains(r"triggers (one more|an additional) time",regex=True)==True)
                                            )
                                            ,1,0)

set_df_kw['untapper'] = np.where(
                                (
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"untap (target|that|another|the chosen|them|all)",regex=True)==True)
                                )
                                &
                                (
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"gain control", regex=True)==False)
                                )
                                ,1,0)

set_df_kw['static_effects'] = np.where(
                                        (
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"(artifacts and creatures|creatures|permanents) (your opponents|enchanted player|you) (control|controls) (enter|lose|have|with|can't)",regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"activated abilities of (artifacts|creatures).*?can't be activated",regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"can't cause their controller to search their library",regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"don't cause abilities to trigger",regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"can't draw more than",regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"only any time they could cast a sorcery",regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"enchanted player",regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"at the beginning of (your|each).*?(you|that player)",regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"(players|counters) can't",regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"if (you|target opponent|a player|another player) would.*?instead",regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"each (card|(.*?) card) in your (hand|graveyard).*?has",regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"(each player|players|your opponents) can't cast (spells|more than)",regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"is determined by their (power|toughness) rather than their (power|toughness)",regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"each creature.*?assigns combat damage.*?toughness rather than its power",regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"they put half that many",regex=True)==True)
                                        )
                                        ,1,0)

set_df_kw['damage_multipliers'] = np.where(
                                            (
                                            (set_df_kw['oracle_text'].str.lower().str.contains(r"it deals that much damage plus",regex=True)==True) |
                                            (set_df_kw['oracle_text'].str.lower().str.contains(r"it deals (double|triple) that damage",regex=True)==True)
                                            )
                                            ,1,0)


set_df_kw['agressive'] = np.where(
                                    (set_df_kw['type_line'].str.lower().str.contains("creature")) &
                                    (
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"(haste|riot|dash)",regex=True)==True)
                                    )
                                    ,1,0)

set_df_kw['doublers'] = np.where(
                                (
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"(put|it creates|it puts|create) twice that many",regex=True)==True)
                                )
                                ,1,0)

set_df_kw['blinker'] = np.where(
                                (
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"exile (up to (one|two) target|up to (one|two) other target|target|another target|any number of target) (creature|creatures|(.*?) creature|permanent|permanents|(.*?) permanent|(.*?) or creature).*?return.*?to the battlefield",regex=True)==True) |
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"exile (target|another target) (permanent|creature).*?return (that card|that permanent|it) to the battlefield under its owner's control",regex=True)==True) |
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"exile (two|three|four|five|all|each).*?you (control|own).*?then return (them|those).*?to the battlefield",regex=True)==True)
                                )
                                ,1,0)

set_df_kw['graveyard_tutor'] = np.where(
                                        (set_df_kw['ramp']!=1) &
                                        (set_df_kw['tutor']!=1) &
                                        (set_df_kw['face_type']!="Land") &
                                        (set_df_kw['face_type']!="Snow Land") &
                                        (set_df_kw['face_type']!="Artifact Land") &
                                        (
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"search your library for.*?put.*?into your graveyard", regex=True)==True)
                                        )
                                        ,1,0)

set_df_kw['play_toplibrary'] = np.where(
                                        (
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"play with the top of your library",regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"you may (play|cast).*?(from the|the) top of your library",regex=True)==True)
                                        )
                                        ,1,0)

set_df_kw['life_lose'] = np.where(
                                    (
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"(each opponent|each player|target opponent|target player).*?loses (.*?) life",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"(afflict|extort)",regex=True)==True)
                                    )
                                    ,1,0)

set_df_kw['play_from_graveyard'] = np.where(
                                            (
                                            (set_df_kw['oracle_text'].str.lower().str.contains(r"you may (play|cast).*?(land|permanent|creature|artifact).*?from your graveyard",regex=True)==True)
                                            )
                                            ,1,0)

set_df_kw['infect'] = np.where(
                                (
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"infect",regex=True)==True)
                                )
                                ,1,0)

set_df_kw['disenchant'] = np.where(
                                    (
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"(destroy|exile) (target|each|every) (artifact or enchantment|artifact|enchantment)",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"destroy (x) target (artifacts or enchantments|artifacts|enchantments)",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"destroy all (artifacts or enchantments|artifacts|enchantments)",regex=True)==True)
                                    )
                                    ,1,0)

set_df_kw['venture'] = np.where(
                                (
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"venture into the dungeon",regex=True)==True)
                                )
                                ,1,0)

set_df_kw['animator'] = np.where(
                                (
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"(target|another target).*?becomes a.*?creature",regex=True)==True)
                                )
                                &
                                (
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"copy", regex=True)==False) &
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"class", regex=True)==False))
                                ,1,0)

set_df_kw['wish'] = np.where(
                            (
                            (set_df_kw['oracle_text'].str.lower().str.contains(r"you may.*?from outside the game",regex=True)==True) |
                            (set_df_kw['oracle_text'].str.lower().str.contains(r"learn",regex=True)==True)
                            )
                            ,1,0)

set_df_kw['gy_synergies'] = np.where(
                                    (
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"gets.*?for each.*?in your graveyard",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"(dredge)",regex=True)==True)
                                    )
                                    ,1,0)

set_df_kw['looting_similar'] = np.where(
                                        (
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"draw (a|one|two|three|four) (card|cards), then discard (a|one|two|three|four) (card|cards)",regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"discard (a|one|two|three|four) (card|cards)(,|:) (draw|then draw) (a|one|two|three|four) (card|cards)",regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"create (.*?) (blood|clue) token",regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"cycling",regex=True)==True)
                                        )
                                        ,1,0)

set_df_kw['cheatinto_play'] = np.where(
                                        (
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"creature.*?put (it|them) onto the battlefield",regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"look at the.*?put.*?creature.*?onto the battlefield",regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"you may put.*?(creature|permanent).*?onto the battlefield",regex=True)==True)
                                        )
                                        ,1,0)

set_df_kw['pumped_foreach'] = np.where(
                                        (
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"gets \+[0-9]/\+[0-9] for each",regex=True)==True)
                                        )
                                        ,1,0)

set_df_kw['ritual'] = np.where(
                                (
                                (set_df_kw['type_line'].str.lower().str.contains("instant")) |
                                (set_df_kw['type_line'].str.lower().str.contains("sorcery"))
                                ) &
                                (
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"add {(.*?)}",regex=True)==True) |
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"add (.*?) {(.*?)}",regex=True)==True)
                                )
                                ,1,0)

set_df_kw['no_maximum'] = np.where(
                                    (
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"you have no maximum hand size",regex=True)==True)
                                    )
                                    ,1,0)

set_df_kw['wheel'] = np.where(
                                (
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"each player.*?(discards|shuffles (his or her|their) hand and graveyard into (his or her|their) library).*?then draws seven cards",regex=True)==True)
                                )
                                ,1,0)

set_df_kw['extra_combat'] = np.where(
                                    (
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"additional combat phase",regex=True)==True)
                                    )
                                    ,1,0)

set_df_kw['ghostly_prison'] = np.where(
                                        (
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"creatures can't attack (you|you or planeswalkers you control) unless",regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"whenever an opponent attacks (you|with creatures)",regex=True)==True)
                                        )
                                        ,1,0)

set_df_kw['land_destruction'] = np.where(
                                        (
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"destroy target (land|nonbasic land)",regex=True)==True) |
                                        (set_df_kw['oracle_text'].str.lower().str.contains(r"destroy all lands",regex=True)==True)
                                        )
                                        ,1,0)

set_df_kw['win_game'] = np.where(
                                (
                                (set_df_kw['oracle_text'].str.lower().str.contains(r"you win the game",regex=True)==True)
                                )
                                ,1,0)

set_df_kw['lose_game'] = np.where(
                                    (
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"you lose the game",regex=True)==True)
                                    )
                                    ,1,0)

set_df_kw['cant_lose'] = np.where(
                                    (
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"you can't lose the game",regex=True)==True) |
                                    (set_df_kw['oracle_text'].str.lower().str.contains(r"your opponents can't win the game",regex=True)==True)
                                    ) ,1,0)

In [ ]:
drop_col = ["name", "scryfall_id", "colors", "keywords", "type_line", "set_name", "set"]

In [60]:
def handle_type(x):
    new_type = ""
    if "//" in x:
        new_type = x.split("//")[0]
    else:
        new_type = x
    
    return new_type

In [71]:
def one_hot_expand(col, prefix, sep=","):
    df2 = df[col].str.split(sep, expand=True)
    stacked = df2.stack()
    one_hot_encoded = pd.get_dummies(stacked, prefix=prefix).groupby(level=0).sum()
    return df.join(one_hot_encoded)

In [61]:
df["card_type"] = df["card_type"].apply(handle_type)

In [62]:
df["card_type"].unique()

array(['Creature', 'Artifact', 'Sorcery', 'Enchantment', 'Planeswalker',
       'Artifact Creature', 'Land', 'Instant', 'Artifact ', 'Land ',
       'Enchantment ', 'Enchantment Creature', 'Snow Creature',
       'Instant ', 'Snow Instant', 'Sorcery ', 'Snow Artifact', 'Snow',
       'Snow Enchantment', 'Snow Sorcery', 'Snow Land', 'Artifact Land',
       'Basic Snow Land'], dtype=object)

In [69]:
df = one_hot_expand("card_type", "card_type", " ")

In [72]:
df = pd.get_dummies(df, columns=['rarity'])

In [74]:
bin = [0,3,6,9,13]

In [75]:
df["target"] = pd.cut(df['avg_pick'], bins=bin)

In [77]:
df['target'] = df['target'].cat.codes

In [79]:
df["target"].value_counts()

-1    4177
 3     429
 2     399
 1     381
 0     328
Name: target, dtype: int64

In [81]:
df["avg_pick"].head(50)

0      0.000000
1      4.181429
2      2.379310
3      0.000000
4      4.980769
5      9.537622
6      0.000000
7     12.630495
8      7.878344
9      0.000000
10     0.000000
11     1.545455
12     3.485370
13     1.306364
14     0.000000
15     7.329591
16     5.030011
17     0.000000
18     7.163480
19     0.000000
20     0.000000
21     1.375203
22     8.620963
23     0.000000
24     4.958002
25     0.000000
26     0.000000
27    11.275701
28     7.280433
29     0.000000
30    10.619389
31     0.000000
32     0.000000
33     0.000000
34    10.229760
35     6.126824
36     1.097001
37     0.000000
38     7.888426
39     0.000000
40     0.000000
41     3.456522
42     0.000000
43     2.994092
44     9.855378
45     0.000000
46     0.000000
47     0.000000
48     0.000000
49     0.000000
Name: avg_pick, dtype: float64